In [2]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import string
import smtplib

import os
from dotenv import find_dotenv, load_dotenv

In [3]:
URL = "https://www.amazon.com/dp/B075CYMYK6?ref_=cm_sw_r_cp_ud_ct_FM9M699VKHTT47YD50Q6&th=1"

headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en-GB,en;q=0.9,ja-JP;q=0.8,ja;q=0.7,en-US;q=0.6",
    #Host: "httpbin.org",
    "Priority": "u=0, i",
    #"Sec-Ch-Ua": "Not)A;Brand";v="8", "Chromium";v="138", "Google Chrome";v="138",
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": "Windows",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "cross-site",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    #"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
    #X-Amzn-Trace-Id: "Root=1-6874ff89-65e6cc15727076c9189f8ec6"
    "user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}
response = requests.get(URL, headers=headers)

In [4]:
raw_web_page = response.text #mengeluarkan html beserta semua tag2 nya
raw_web_page

'<!doctype html><html lang="en-us" class="a-no-js" data-19ax5a9jf="dingo"><!-- sp:feature:head-start -->\n<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>\n<!-- sp:end-feature:head-start -->\n<!-- sp:feature:csm:head-open-part1 -->\n\n<script type=\'text/javascript\'>var ue_t0=ue_t0||+new Date();</script>\n<!-- sp:end-feature:csm:head-open-part1 -->\n<!-- sp:feature:cs-optimization -->\n<meta http-equiv=\'x-dns-prefetch-control\' content=\'on\'>\n<link rel="dns-prefetch" href="https://images-na.ssl-images-amazon.com">\n<link rel="dns-prefetch" href="https://m.media-amazon.com">\n<link rel="dns-prefetch" href="https://completion.amazon.com">\n<!-- sp:end-feature:cs-optimization -->\n<!-- sp:feature:csm:head-open-part2 -->\n<script type=\'text/javascript\'>\nwindow.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;\nif (window.ue_ihb === 1) {\n\nvar ue_csm = window,\n    ue_hob = +new Date();\n(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){

In [5]:
soup = BeautifulSoup(raw_web_page, "html.parser") #membuat object untuk parsing html

In [6]:
product_title = soup.find(name = "span", id = "productTitle")
product_title =product_title.get_text()
product_title = product_title.split("\r\n                                        ")
product_title = " ".join(product_title)
product_title = product_title.strip()
product_title

'Instant Pot Duo Plus 9-in-1 Electric Pressure Cooker, Slow Cooker, Rice Cooker, Steamer, Sauté, Yogurt Maker, Warmer & Sterilizer, Includes App With Over 800 Recipes, Stainless Steel, 3 Quart'

In [9]:
price_tag = soup.find(name = "span", class_ = "aok-offscreen")
price_tag = price_tag.get_text()
price_tag = price_tag.strip()
price_tag = price_tag[1:]
price_tag = price_tag.split()
price_tag = price_tag[0]
price_tag = float(price_tag)
price_tag

99.99

# Mengirim email dan menyimpan credentials di env variabel

In [10]:
# mencari lokasi file .env secara otomatis 
dotenv_path = find_dotenv()

# load the entries as environtment variables
load_dotenv(dotenv_path)

True

In [11]:
# stored the env variables within a python variable
my_email = os.getenv("my_email")
my_password = os.getenv("my_password")
email_penerima = os.getenv("email_imam")

In [12]:
my_password

'otlcxftlknqbeype'

In [13]:
treshold = 100.00

if price_tag < treshold:
    # send an email

    with smtplib.SMTP("smtp.gmail.com", 587) as connection:
        connection.starttls() #TLS : transport layer security
        connection.login(user = my_email, password = my_password)
        connection.sendmail(from_addr = my_email, 
                            to_addrs= email_penerima,
                            msg=f"Subject:Amazon Price Alert!\n\n{product_title} is now ${price_tag}".encode("utf-8"))

else:
    print(f"Harga belum turun di bawah {treshold}")